In [39]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [40]:
path = 'kc_house_data\kc_house_data.csv'

data = pd.read_csv(path)
data.head()

<>:1: SyntaxWarning: invalid escape sequence '\k'
<>:1: SyntaxWarning: invalid escape sequence '\k'
C:\Users\srinivas\AppData\Local\Temp\ipykernel_9960\4216559414.py:1: SyntaxWarning: invalid escape sequence '\k'
  path = 'kc_house_data\kc_house_data.csv'


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [41]:
X = data.drop('price',axis=1)
y = data['price']
X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X.head()
X.describe()
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17290 entries, 6325 to 15795
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             17290 non-null  int64  
 1   date           17290 non-null  object 
 2   bedrooms       17290 non-null  int64  
 3   bathrooms      17290 non-null  float64
 4   sqft_living    17290 non-null  int64  
 5   sqft_lot       17290 non-null  int64  
 6   floors         17290 non-null  float64
 7   waterfront     17290 non-null  int64  
 8   view           17290 non-null  int64  
 9   condition      17290 non-null  int64  
 10  grade          17290 non-null  int64  
 11  sqft_above     17290 non-null  int64  
 12  sqft_basement  17290 non-null  int64  
 13  yr_built       17290 non-null  int64  
 14  yr_renovated   17290 non-null  int64  
 15  zipcode        17290 non-null  int64  
 16  lat            17290 non-null  float64
 17  long           17290 non-null  float64
 18  sqft_liv

In [42]:
X_train_full,X_valid_full,y_train,y_test = train_test_split(X,y,train_size=0.8,test_size=0.2,random_state=0)

In [43]:
categorical_col=[col for col in X_train_full.columns if X_train_full[col].dtype=='O' and X_train_full[col].nunique()<10]
print(categorical_col)

numerical_col=[col for col in X_train_full.columns if X_train_full[col].dtype in ['float64','int64']]
print(numerical_col)

my_cols=categorical_col + numerical_col
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

[]
['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [44]:
X_train.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
1565,4215250310,4,2.50,3720,35000,2.0,0,0,3,10,3720,0,1983,0,98072,47.7582,-122.130,3720,35000
8884,8122100835,2,1.00,670,5140,1.0,0,0,3,6,670,0,1926,0,98126,47.5387,-122.371,850,5140
14985,8665900168,4,2.50,4260,36360,1.5,0,0,4,8,4100,160,1935,0,98155,47.7640,-122.302,2430,17888
8021,6806100040,4,2.50,2000,5006,2.0,0,0,3,7,2000,0,2005,0,98058,47.4660,-122.147,2410,4889
6432,492000532,4,2.75,2420,8700,1.5,0,0,4,7,2420,0,1914,0,98002,47.3109,-122.229,1070,4804


In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
                                          ('impute',SimpleImputer(strategy='most_frequent')),
                                          ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
                                               ('num',numerical_transformer,numerical_col),
                                               ('cat',categorical_transformer,categorical_col)
])

In [46]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
eval_set_pipe = Pipeline(steps = [('preprocessor', preprocessor)])

In [47]:
X_train_eval=X_train.copy()
X_train_eval = eval_set_pipe.fit_transform(X_train)
pd.DataFrame(X_train_eval,index=X_train.index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1565,4.215250e+09,4.0,2.50,3720.0,35000.0,2.0,0.0,0.0,3.0,10.0,3720.0,0.0,1983.0,0.0,98072.0,47.7582,-122.130,3720.0,35000.0
8884,8.122101e+09,2.0,1.00,670.0,5140.0,1.0,0.0,0.0,3.0,6.0,670.0,0.0,1926.0,0.0,98126.0,47.5387,-122.371,850.0,5140.0
14985,8.665900e+09,4.0,2.50,4260.0,36360.0,1.5,0.0,0.0,4.0,8.0,4100.0,160.0,1935.0,0.0,98155.0,47.7640,-122.302,2430.0,17888.0
8021,6.806100e+09,4.0,2.50,2000.0,5006.0,2.0,0.0,0.0,3.0,7.0,2000.0,0.0,2005.0,0.0,98058.0,47.4660,-122.147,2410.0,4889.0
6432,4.920005e+08,4.0,2.75,2420.0,8700.0,1.5,0.0,0.0,4.0,7.0,2420.0,0.0,1914.0,0.0,98002.0,47.3109,-122.229,1070.0,4804.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5822,3.361401e+09,2.0,1.00,1130.0,8160.0,1.0,0.0,0.0,4.0,6.0,1130.0,0.0,1952.0,0.0,98168.0,47.4997,-122.317,1060.0,6120.0
10335,7.942300e+09,4.0,1.75,1550.0,9899.0,1.0,0.0,0.0,4.0,7.0,1550.0,0.0,1967.0,0.0,98030.0,47.3583,-122.183,1500.0,11272.0
6073,2.925059e+09,5.0,2.50,3000.0,10560.0,1.0,0.0,0.0,3.0,8.0,1500.0,1500.0,1966.0,0.0,98004.0,47.6249,-122.206,2690.0,11616.0
21474,2.895801e+09,5.0,2.50,2170.0,2752.0,2.0,0.0,0.0,3.0,8.0,2170.0,0.0,2014.0,0.0,98106.0,47.5167,-122.347,1800.0,2752.0


In [48]:


# Make a copy to avoid changing original data
X_valid_eval=X_valid.copy()
# fit transform X_valid.copy()
X_valid_eval = eval_set_pipe.transform (X_valid_eval)

pd.DataFrame(X_valid_eval, index=X_valid.index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
4527,1.545806e+09,3.0,1.75,1590.0,7620.0,1.0,0.0,0.0,4.0,7.0,1090.0,500.0,1984.0,0.0,98038.0,47.3655,-122.048,1590.0,7620.0
11503,9.294300e+09,4.0,2.00,1820.0,5000.0,1.5,0.0,1.0,3.0,7.0,1640.0,180.0,1945.0,0.0,98115.0,47.6815,-122.269,2060.0,5000.0
10565,8.078460e+09,3.0,2.50,2140.0,8925.0,2.0,0.0,0.0,3.0,8.0,2140.0,0.0,1991.0,0.0,98074.0,47.6314,-122.027,2310.0,8956.0
5380,6.421101e+09,5.0,3.00,3290.0,14134.0,1.0,0.0,0.0,3.0,7.0,1870.0,1420.0,2004.0,0.0,98052.0,47.6708,-122.140,1970.0,11470.0
20449,3.679400e+09,3.0,1.75,1420.0,1361.0,2.0,0.0,0.0,3.0,7.0,960.0,460.0,2014.0,0.0,98108.0,47.5684,-122.314,1340.0,1343.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12430,5.100403e+09,3.0,1.00,860.0,6380.0,1.5,0.0,0.0,4.0,7.0,860.0,0.0,1927.0,0.0,98115.0,47.6942,-122.315,1250.0,6380.0
8789,3.256400e+09,3.0,2.00,960.0,9380.0,1.0,0.0,0.0,3.0,6.0,960.0,0.0,1949.0,0.0,98146.0,47.4852,-122.343,1460.0,9240.0
13277,1.402650e+09,3.0,2.50,2430.0,7613.0,2.0,0.0,0.0,4.0,8.0,2430.0,0.0,1986.0,0.0,98058.0,47.4383,-122.135,2440.0,8342.0
1960,6.430500e+09,4.0,1.50,1790.0,3952.0,1.5,0.0,0.0,4.0,8.0,1790.0,0.0,1932.0,0.0,98103.0,47.6890,-122.352,1200.0,3876.0


In [49]:
my_model.fit(X_train_eval, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

In [50]:
from sklearn.metrics import mean_absolute_error
preds = my_model.predict(X_valid_eval)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)


MAE: 66297.07376463996


In [54]:
from sklearn.metrics import mean_squared_error
X_test_full=X_test
X_test=X_test_full[my_cols].copy()

X_test_eval=X_test.copy()
X_test_eval = eval_set_pipe.transform (X_test_eval)
pd.DataFrame(X_test_eval, index=X_test_full.index)

pred_test=my_model.predict(X_test_eval)

ValueError: Found input variables with inconsistent numbers of samples: [3458, 4323]